In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score

In [2]:
import opendatasets as od

In [3]:
dataset= 'https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset?select=Fake.csv'

In [4]:
od.download(dataset)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 41.0M/41.0M [00:12<00:00, 3.54MB/s]


In [ ]:
df = pd.read_csv('train.csv')
conversion_dict = {0: 'Real', 1: 'Fake'}
df['label'] = df['label'].replace(conversion_dict)
df.label.value_counts()

In [ ]:
df.head(5)

In [ ]:
df.isna().sum()

In [ ]:
df.fillna("missing",inplace = True)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df['text'], df['label'], test_size=0.25, random_state=7, shuffle=True)
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.75)

In [ ]:
vec_train=tfidf_vectorizer.fit_transform(x_train.values.astype("U"))
vec_test=tfidf_vectorizer.transform(x_test.values.astype("U"))

In [ ]:
pac=MultinomialNB(max_iter=50)
pac.fit(vec_train,y_train)

In [ ]:
y_pred = pac.predict(vec_test)
score = accuracy_score(y_test, y_pred)
print(f"PAC Accuracy: {round(score*100,2)}%")

In [ ]:
X = tfidf_vectorizer.transform(df['text'].values.astype('U'))

In [ ]:
scores = cross_val_score(pac, X, df['label'].values, cv=5)
print(f"K Fold Accuracy: {round(scores.mean()*100,2)}%")

In [ ]:
df_true = pd.read_csv('True.csv')
df_true['label'] = 'Real'
df_true_rep = [df_true['text'][i].replace('WASHINGTON (Reuters) - ', '').replace('LONDON (Reuters) - ', '').replace('(Reuters) - ', '') for i in range(len(df_true['text']))]
df_true['text'] = df_true_rep
df_fake = pd.read_csv('Fake.csv')
df_fake['label'] = 'Fake'
df_final = pd.concat([df_true, df_fake])
df_final = df_final.drop(['subject', 'date'], axis=1)
df_fake

In [ ]:
def findlabel(newtext):
    vec_newtest = tfidf_vectorizer.transform([newtext])
    y_pred1 = pac.predict(vec_newtest)
    return y_pred1[0]

In [ ]:
findlabel((df_true['text'][0]))

In [ ]:
findlabel((df_fake['text'][0]))

In [ ]:
sum([1 if findlabel((df_true['text'][i])) == 'Real' else 0 for i in range(len(df_true['text']))]) / df_true['text'].size

In [ ]:
sum([1 if findlabel((df_fake['text'][i])) == 'Fake' else 0 for i in range(len(df_fake['text']))]) / df_fake['text'].size

In [ ]:
import joblib

In [ ]:
joblib.dump(pac, 'model.sav')

In [ ]:
joblib.dump(tfidf_vectorizer, 'vectorizer.sav')